In [ ]:


import numpy as np
import tensorflow as tf
import pathlib
from pathlib import Path
import librosa
import os


file_list = None
train_ds = None
val_ds = None
test_ds = None
label_names = None
example_audio = None
example_labels = None
example_filenames = None



TRAIN_DIR = None
TEST_DIR = None
DATA_DIR = None
VAL_DIR = None


Divided into directories this way, you can easily load the data using `keras.utils.audio_dataset_from_directory`.

The audio clips are 1 second or less at 16kHz. The `output_sequence_length=16000` pads the short ones to exactly 1 second (and would trim longer ones) so that they can be easily batched.


In [ ]:
def build():
    global file_list
    global train_ds
    global val_ds
    global label_names
    file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)

    seconds=40
    train_ds = tf.keras.utils.audio_dataset_from_directory(
        directory=TRAIN_DIR,
        batch_size=64,
        seed=0,
        output_sequence_length=16000*seconds,
        ) 
    val_ds = tf.keras.utils.audio_dataset_from_directory(
        directory=VAL_DIR,
        batch_size=64,
        seed=0,
        output_sequence_length=16000*seconds,
        ) 

    label_names = np.array(train_ds.class_names)
    # set_label_names(np.array(train_ds.class_names))
    print()
    print("train_ds label names:", label_names)
    print("val_ds label names:  ", np.array(val_ds.class_names))
    
    #The dataset now contains batches of audio clips and integer labels. 
    #The audio clips have a shape of `(batch, samples, channels)`.
    train_ds.element_spec

This dataset only contains single channel audio, so use the `tf.squeeze` function to drop the extra axis:


In [ ]:
def drop():
  global train_ds
  global val_ds
  @tf.autograph.experimental.do_not_convert
  def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

  train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
  val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

The `utils.audio_dataset_from_directory` function only returns up to two splits. It's a good idea to keep a test set separate from your validation set.
Ideally you'd keep it in a separate directory, but in this case you can use `Dataset.shard` to split the validation set into two halves. Note that iterating over **any** shard will load **all** the data, and only keep its fraction.


In [ ]:
def split():
    global test_ds
    global val_ds
    test_ds = val_ds.shard(num_shards=2, index=0)
    val_ds = val_ds.shard(num_shards=2, index=1)

In [ ]:
def extract():
  global example_audio
  global example_labels
  global example_filenames
  for example_audio, example_labels in train_ds.take(1):  
    print("Audio Shape:", example_audio.shape)
    print("Labels Shape:", example_labels.shape)
    print("Batch Labels:", example_labels.numpy())
    
    # Extrahiere Dateinamen aus dem Dataset
  example_filenames = []
  for filepath in file_list.take(len(example_audio)):
      example_filenames.append(pathlib.Path(filepath.numpy().decode('utf-8')).name)

In [ ]:
def compare():
    global example_filenames
    file_labels = []
    example_filenames = []
    for filepath in file_list.take(len(example_audio)):
        example_filenames.append(pathlib.Path(filepath.numpy().decode('utf-8')).name)
    print(f"example filenames: {example_filenames}")
    for dirpath, dirnames, filenames in os.walk(TRAIN_DIR):
        for filename in filenames:
            if filename.endswith('.wav'):
                label = filename.split('_')[0]
                file_labels.append(label)

    # Erstelle ein Mapping für die Labels
    label_mapping = {
        'orig-16-44-mono': 1,
        'upscale-from-mp3-128': 0,
    }

    # Erstelle eine Liste der gemappten Labels
    mapped_labels = [label_mapping.get(label, -1) for label in file_labels]
    print("Gemappte Labels aus Dateinamen:", mapped_labels)

    # Vergleiche die gemappten Labels mit den Batch-Labels
    for i in range(min(len(mapped_labels), len(example_labels.numpy()))):
        print(f"Index: {i}, Batch Label: {example_labels.numpy()[i]}, Gemapptes Label: {mapped_labels[i]}")

In [ ]:
def run(_TRAIN_DIR, _TEST_DIR, _DATA_DIR, _VAL_DIR):
    global TRAIN_DIR, TEST_DIR, DATA_DIR,VAL_DIR, file_list, example_labels, example_audio, train_ds, val_ds, test_ds, label_names, example_audio, example_labels, example_filenames

    TRAIN_DIR =_TRAIN_DIR
    TEST_DIR =_TEST_DIR
    DATA_DIR = _DATA_DIR
    VAL_DIR = _VAL_DIR

    build()
    drop()
    split()
    extract()
    compare()
    
    return file_list, example_labels, example_audio, train_ds, val_ds, test_ds, label_names, example_audio, example_labels, example_filenames